In [9]:
#!pip install youtube-transcript-api
from langchain_core.runnables import RunnableBranch,RunnableParallel,RunnablePassthrough,RunnableSequence,RunnableLambda 
#using this runnable class futhure we can used in chains.
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate #this class we used to develop structure instruction prompt

#below is the class of llm model where we are doing interation
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint
from langchain_groq import ChatGroq

#using document loaders to load document from multiple source to myy working space
from langchain_community.document_loaders import TextLoader,PDFPlumberLoader,WebBaseLoader

#using text splitter to split the document into chunks.
from langchain.text_splitter import RecursiveCharacterTextSplitter

#importing vector store databse class
from langchain_community.vectorstores import FAISS

#now importing retriever class 
from langchain.retrievers import MultiQueryRetriever,ContextualCompressionRetriever
 
#here we are using youtube transcriber module.
from youtube_transcript_api import YouTubeTranscriptApi

#import output parser class
from langchain_core.output_parsers import StrOutputParser,PydanticOutputParser

from pydantic import BaseModel, Field,computed_field

from typing import Literal,Annotated,TypedDict

from dotenv import load_dotenv

load_dotenv()

True

# step1:- Data Ingestion in RAG it also known as INDEXING

# generating text transcribe from yoututbe video

In [2]:
try:
    #creating an object of YouTubeTranscriptApi class
    video_id = "Gfr50f6ZBvo"
    you_api = YouTubeTranscriptApi()
    fetch_transcript = you_api.fetch(video_id=video_id) #rtn as a list object of transcript from given video id.
    
    
    #using list comphression to get text content only from transcript.
    text = "\n".join([value.text  if value.text!="" else "\n" for i,value in enumerate(fetch_transcript)])
    
    #above text file we write into textual filte
    with open(file="transcript_demis.txt",mode="w", encoding="utf-8") as file:
        file.write(text)
    print("transcript write into textual file")  

    
except Exception as e:
    print(e)

transcript write into textual file


# step:2

# using document loader class we can load the text file into my working space

In [3]:
#creating an object of Textloader class
text_loader = TextLoader(
    file_path=r"transcript_demis.txt",
    encoding="utf-8"
    
)
text_loader

In [4]:
#now loading the textual document to my working space.
document = text_loader.load() #rtn as list of document
print(document)

[Document(metadata={'source': 'transcript_demis.txt'}, page_content="the following is a conversation with\ndemus hasabis\nceo and co-founder of deepmind\na company that has published and builds\nsome of the most incredible artificial\nintelligence systems in the history of\ncomputing including alfred zero that\nlearned\nall by itself to play the game of gold\nbetter than any human in the world and\nalpha fold two that solved protein\nfolding\nboth tasks considered nearly impossible\nfor a very long time\ndemus is widely considered to be one of\nthe most brilliant and impactful humans\nin the history of artificial\nintelligence and science and engineering\nin general\nthis was truly an honor and a pleasure\nfor me to finally sit down with him for\nthis conversation and i'm sure we will\ntalk many times again in the future\nthis is the lex friedman podcast to\nsupport it please check out our sponsors\nin the description and now dear friends\nhere's demis\nhassabis\nlet's start with a bit

In [5]:
for i,doc in enumerate(document):
    print('index of document: ',i+1)
    print(doc.page_content)

index of document:  1
the following is a conversation with
demus hasabis
ceo and co-founder of deepmind
a company that has published and builds
some of the most incredible artificial
intelligence systems in the history of
computing including alfred zero that
learned
all by itself to play the game of gold
better than any human in the world and
alpha fold two that solved protein
folding
both tasks considered nearly impossible
for a very long time
demus is widely considered to be one of
the most brilliant and impactful humans
in the history of artificial
intelligence and science and engineering
in general
this was truly an honor and a pleasure
for me to finally sit down with him for
this conversation and i'm sure we will
talk many times again in the future
this is the lex friedman podcast to
support it please check out our sponsors
in the description and now dear friends
here's demis
hassabis
let's start with a bit of a personal
question
am i an ai program you wrote to
interview people un

# now using text splitting technique we r splitting these large textual documents to chunks

In [6]:
#creating an object of recursiveTEXT splitter class
splitter_obj = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)
splitter_obj

In [7]:
#now splitting the text file into chunks
lst_chunks = splitter_obj.split_documents(document)
len(lst_chunks)

169

# step3:- converting those chunks file to embedding vector and storing them vector DB

In [8]:
# from langchain_huggingface import HuggingFaceEmbeddings

# import os
# os.environ["HF_HOME"] = "D:/huggingface_cache" downloading to this location

# model_name = "Qwen/Qwen3-Embedding-8B"
# model_kwargs = {'device': 'cuda'}
# encode_kwargs = {'normalize_embeddings': False}
# hf = HuggingFaceEmbeddings(
#     model_name=model_name,
#     model_kwargs=model_kwargs,
#     encode_kwargs=encode_kwargs
# )
# hf

In [10]:
#using chatgpt embedding model
emb_model = OpenAIEmbeddings(
    model="text-embedding-3-small"
)
emb_model

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000017BF1592F80>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000017BF1DC91E0>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

# step4:- where we are storing embdding vector in vector store 

In [12]:
#creating an object of vector store Database and storing the embedding into it by using from_document method.
faiss_vectore = FAISS.from_documents(
    embedding=emb_model,
    documents=lst_chunks,
)
faiss_vectore

In [13]:
#to show embedding vector get store in vector store or not
print("Number of vectors stored:", faiss_vectore.index.ntotal)

Number of vectors stored: 169


In [16]:
#Ek-ek document ID aur text print karna
for doc_id, doc in faiss_vectore.docstore._dict.items():
    print(f"ID: {doc_id}")
    print(f"Content: {doc.page_content[:200]}...")  # first 200 chars
    print(f"Metadata: {doc.metadata}")
    print("-" * 50)

ID: 5b4ad5ff-e363-405e-88de-a749429b3ef2
Content: the following is a conversation with
demus hasabis
ceo and co-founder of deepmind
a company that has published and builds
some of the most incredible artificial
intelligence systems in the history of
...
Metadata: {'source': 'transcript_demis.txt'}
--------------------------------------------------
ID: 31bf099b-ef0b-445b-b3b5-5bd62668471a
Content: in the description and now dear friends
here's demis
hassabis
let's start with a bit of a personal
question
am i an ai program you wrote to
interview people until i get good enough
to interview you
we...
Metadata: {'source': 'transcript_demis.txt'}
--------------------------------------------------
ID: 5cdc2787-b086-40aa-b290-982fbc010476
Content: benchmark from the future where they
replay 2022 as a year before ais were
good enough yet and now we want to see
is it going to pass exactly
if i was such a
program would you be able to tell do you
t...
Metadata: {'source': 'transcript_demis.txt'}
-

In [ ]:
#to show only those ids which are getting stored in to vector database
faiss_vectore.index_to_docstore_id

{0: '5b4ad5ff-e363-405e-88de-a749429b3ef2',
 1: '31bf099b-ef0b-445b-b3b5-5bd62668471a',
 2: '5cdc2787-b086-40aa-b290-982fbc010476',
 3: 'e354efd7-26e8-45b3-8a63-744e81d1ac2f',
 4: '54942898-59ca-46fd-8b11-0a26c34d82b2',
 5: '82ff9332-931e-438f-a605-9239d37b698f',
 6: '8e56a644-a6bc-489a-9eb3-d6c0bb8d32f7',
 7: '3fc632d3-838e-467b-9fe2-0e4994fae5f9',
 8: '39c1fb55-467e-4aa5-b0d3-e9071d80bf97',
 9: 'd37e4d77-92c4-4c5b-8e01-712de1ae693f',
 10: '8e2ed2ce-3273-455d-9d59-a62161d6313b',
 11: '8e5fd517-2c14-4fc8-bb45-89318032be15',
 12: 'ec289089-233b-4385-a2a0-7fce65997a36',
 13: '875f0e2b-f039-4b93-a2fe-03589d468836',
 14: '24b03ed6-134f-483d-8872-da53e91909de',
 15: 'cdf2eb65-0aa1-40e6-94a2-141c650317ec',
 16: 'b7eee01a-b7bc-4c9c-92f8-72ad4b673402',
 17: 'bab92cda-4aaf-4496-889b-f85a91ff3476',
 18: '49ef0200-5205-4e1d-901f-f73e14d53aa7',
 19: 'a7f0da27-eab1-435f-a6f4-a25e00dbe653',
 20: '1e99b0c7-c56e-4ae4-865f-2bd004e7d914',
 21: '816dcc2a-2182-46ae-ada4-78829399f6b8',
 22: 'e60ec109-9a4e-

# step:5 creating retrievers that will fetch the imformation from database based on user query

In [38]:
chat_model  = ChatGroq(model="DeepSeek-R1-Distill-Llama-70B",
                       temperature=0.2)
chat_model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000017BF37AD8A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000017BF3857E80>, model_name='DeepSeek-R1-Distill-Llama-70B', temperature=0.2, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [39]:
#creating base_retrievers from vector store database
base_retriever = faiss_vectore.as_retriever(
    # Retrieve more documents with higher diversity
    # Useful if your dataset has many similar documents
    search_type="mmr", #which algorithm we are using in searching method mmr(will diversity soln)
    search_kwargs={'k': 4, 'lambda_mult': 0.25}
    
)
base_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000017BF20A14E0>, search_type='mmr', search_kwargs={'k': 4, 'lambda_mult': 0.25})

In [40]:
# Compressor wrapper
from langchain.retrievers.document_compressors import LLMChainExtractor
compressor = LLMChainExtractor.from_llm(chat_model)

In [41]:
# Final compressed retriever
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)


In [42]:
#Testing the query
query = "what is deepmind"
result = compression_retriever.invoke(input=query)
result

[Document(metadata={'source': 'transcript_demis.txt'}, page_content="<think>\nOkay, so I need to figure out what DeepMind is based on the given context. Let me read through the context carefully. \n\nThe context starts by introducing a conversation with Demis Hassabis, who is the CEO and co-founder of DeepMind. It mentions that DeepMind has published and built some of the most incredible AI systems in computing history. Specifically, it talks about AlphaZero, which learned to play the game of Go better than any human, and AlphaFold 2, which solved protein folding. Both of these tasks were considered nearly impossible for a long time. \n\nSo, from this, I can gather that DeepMind is a company focused on developing advanced artificial intelligence systems. They've made significant breakthroughs in AI, as evidenced by their work on AlphaZero and AlphaFold 2. These achievements highlight their expertise in creating AI that can perform complex tasks that were previously thought to be beyond

# step:-6 Augmentation(combination user query + retrieval fetch information)

In [43]:
prompt = PromptTemplate(
    template="""
    you are very helpful assistant.
    answer only from the provided transcript context.
    if the context is insufficient, just say you don't know.
    
    {context}
    Question: {uesr_query}
    """,
    input_variables=['context','uesr_query']
)
prompt

PromptTemplate(input_variables=['context', 'uesr_query'], input_types={}, partial_variables={}, template="\n    you are very helpful assistant.\n    answer only from the provided transcript context.\n    if the context is insufficient, just say you don't know.\n    \n    {context}\n    Question: {uesr_query}\n    ")

# system query

In [ ]:
user_query = "is the topic of aliens discussed in this video? if yes then what was discussed"
retrieve_doc = compression_retriever.invoke(user_query) #rtn as lsit of document

In [45]:
context_text_info = "\n\n".join([doc.page_content for doc in retrieve_doc])
context_text_info

'<think>\nOkay, so I need to figure out if the topic of aliens is discussed in the given video context and, if so, what was discussed. Let me start by reading through the context carefully.\n\nThe context provided is a transcript of someone speaking. They mention "interactions with our mind" that might not originate from us but from "other life forms elsewhere." That immediately makes me think of aliens or extraterrestrial life. So, yes, aliens are being discussed here.\n\nNext, the speaker talks about consciousness and how it might be something that could be used by alien species. They question why all alien species would use this method, suggesting that some might be more primitive, similar to our level, while others could be more advanced. They mention a "normal distribution" of these life forms, implying a variety in their characteristics.\n\nThe speaker also speculates about the behavior of these alien civilizations. They say some could be aggressive, others curious, and some phil

In [46]:
#geenrating augumentation
final_prompt  = prompt.invoke(input={'context':context_text_info,"uesr_query":user_query})
final_prompt

StringPromptValue(text='\n    you are very helpful assistant.\n    answer only from the provided transcript context.\n    if the context is insufficient, just say you don\'t know.\n    \n    <think>\nOkay, so I need to figure out if the topic of aliens is discussed in the given video context and, if so, what was discussed. Let me start by reading through the context carefully.\n\nThe context provided is a transcript of someone speaking. They mention "interactions with our mind" that might not originate from us but from "other life forms elsewhere." That immediately makes me think of aliens or extraterrestrial life. So, yes, aliens are being discussed here.\n\nNext, the speaker talks about consciousness and how it might be something that could be used by alien species. They question why all alien species would use this method, suggesting that some might be more primitive, similar to our level, while others could be more advanced. They mention a "normal distribution" of these life forms,

# step:7 Generation

In [48]:
answer = chat_model.invoke(input=final_prompt)
answer

AIMessage(content='\nOkay, so I need to figure out if the topic of aliens is discussed in the given context. Let me read through the context carefully. \n\nThe context starts with a discussion about the universe, mentioning that it\'s 13.8 billion years old and that we\'re just starting to understand it. It then talks about the possibility of other life forms and the Fermi Paradox, which questions why we haven\'t detected extraterrestrial life yet. The speaker mentions that we might be too primitive to notice advanced civilizations and that they could be using communication methods beyond our understanding, like consciousness itself. They also discuss the diversity of alien species, suggesting that some might be aggressive, others curious, and some philosophical, especially if they\'re millions of years older than us. The speaker argues against the idea that all alien civilizations would be uniform in their behavior or communication methods, giving an example of a violent dictatorship.

In [50]:
answer.content

'\nOkay, so I need to figure out if the topic of aliens is discussed in the given context. Let me read through the context carefully. \n\nThe context starts with a discussion about the universe, mentioning that it\'s 13.8 billion years old and that we\'re just starting to understand it. It then talks about the possibility of other life forms and the Fermi Paradox, which questions why we haven\'t detected extraterrestrial life yet. The speaker mentions that we might be too primitive to notice advanced civilizations and that they could be using communication methods beyond our understanding, like consciousness itself. They also discuss the diversity of alien species, suggesting that some might be aggressive, others curious, and some philosophical, especially if they\'re millions of years older than us. The speaker argues against the idea that all alien civilizations would be uniform in their behavior or communication methods, giving an example of a violent dictatorship.\n\nSo, yes, the t

# chains forming

In [51]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieve_doc)
  return context_text

In [57]:
parallel_chain = RunnableParallel({
    'context': compression_retriever | RunnableLambda(format_docs),
    'uesr_query': RunnablePassthrough()
})
parallel_chain

{
  context: ContextualCompressionRetriever(base_compressor=LLMChainExtractor(llm_chain=PromptTemplate(input_variables=['context', 'question'], input_types={}, output_parser=NoOutputParser(), partial_variables={}, template='Given the following question and context, extract any part of the context *AS IS* that is relevant to answer the question. If none of the context is relevant return NO_OUTPUT.\n\nRemember, *DO NOT* edit the extracted parts of the context.\n\n> Question: {question}\n> Context:\n>>>\n{context}\n>>>\nExtracted relevant parts:')
           | ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000017BF37AD8A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000017BF3857E80>, model_name='DeepSeek-R1-Distill-Llama-70B', temperature=0.2, model_kwargs={}, groq_api_key=SecretStr('**********'))
           | NoOutputParser(), get_input=<function default_get_input at 0x0000017BF5571C60>), base_retriever=VectorStoreRetriever(tags

In [58]:
parallel_chain.invoke('who is Demis')

{'context': '<think>\nOkay, so I need to figure out if the topic of aliens is discussed in the given video context and, if so, what was discussed. Let me start by reading through the context carefully.\n\nThe context provided is a transcript of someone speaking. They mention "interactions with our mind" that might not originate from us but from "other life forms elsewhere." That immediately makes me think of aliens or extraterrestrial life. So, yes, aliens are being discussed here.\n\nNext, the speaker talks about consciousness and how it might be something that could be used by alien species. They question why all alien species would use this method, suggesting that some might be more primitive, similar to our level, while others could be more advanced. They mention a "normal distribution" of these life forms, implying a variety in their characteristics.\n\nThe speaker also speculates about the behavior of these alien civilizations. They say some could be aggressive, others curious, a

In [59]:
parser = StrOutputParser()
main_chain = parallel_chain | prompt | chat_model | parser

In [60]:
main_chain.invoke("can you summarize the video")

"<think>\nOkay, so I need to summarize the video based on the provided context. Let me read through the context again to understand what it's about.\n\nThe context starts with a discussion about tools and how they might not help us understand everything. It mentions non-human systems possibly reaching farther, not just as tools but in themselves understanding something they can bring back. Then it goes into a conversation about what humans can understand today versus the totality of what's there to be understood. It talks about three consensus points, exploring more branches of a tree, and how AI will help explore that. It ends with a question about the totality of what could be understood and mentions fundamental physics reasons why certain things can't be understood, like what's outside the universe.\n\nSo, the video seems to be discussing the limitations of human understanding and how AI might help expand our knowledge. It touches on the idea that there are things beyond our current